공공데이타 - https://www.data.go.kr/dataset/15012005/fileData.do

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("상가정보_서울_202403.csv", dtype={2: str})

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# 컬럼 정리
columns = ['상호명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '법정동명', '도로명주소', 
           '경도', '위도']
df = df[columns].copy()
df.shape

In [ ]:
df["시군구명"].unique()

In [ ]:
df['상호명_소문자'] = df['상호명'].str.lower()

In [ ]:
df.columns

In [ ]:
df_31 = df[df["상호명_소문자"].str.contains('배스킨라빈스|베스킨라빈스|baskinrobbins|던킨|dunkin')].copy()

In [ ]:
df_31.shape

In [ ]:
df_31.loc[df_31["상호명_소문자"].str.contains("배스킨라빈스|베스킨라빈스|baskinrobbins"), "브랜드명"] = "배스킨라빈스"
df_31.loc[df_31["상호명_소문자"].str.contains("던킨|dunkin"), "브랜드명"] = "던킨도너츠"

In [ ]:
df_31[["상호명", "브랜드명"]].head()

In [ ]:
df_31.columns

In [ ]:
df_31 = df_31[df_31['상권업종대분류명'] == '음식']

In [ ]:
df_31.shape

In [ ]:
brand_count = df_31["브랜드명"].value_counts() # "브랜드명"의 빈도수
brand_count

In [ ]:
relative_frequency = df_31["브랜드명"].value_counts(normalize=True)  # normalize=True 빈도수의 비율
relative_frequency

In [ ]:
plt.rc('font', family='Malgun Gothic') 

In [ ]:
relative_frequency.plot.barh();

In [ ]:
g = sns.countplot(data=df_31, x="브랜드명")

for i, val in enumerate(brand_count):
    g.text(x=i, y=val, s=val)


In [ ]:
plt.figure(figsize=(15, 4))
g = sns.countplot(data=df_31, x="시군구명", hue="브랜드명")

In [ ]:
table_city_brand = df_31.pivot_table(index="시군구명", columns="브랜드명", values="상호명", aggfunc="count")
table_city_brand.head()

In [ ]:
df_31[["위도", "경도"]].plot.scatter(x="경도", y="위도");   # 산점도(scatter plot)

In [ ]:
sns.scatterplot(data=df_31, x="경도", y="위도", hue="브랜드명");

https://python-visualization.github.io/folium/latest/user_guide/map.html

지도 시각화를 위한 라이브러리   
pip install folium

In [ ]:
import folium

In [ ]:
# 지도의 중심을 지정하기 위해 위도와 경도의 평균을 구합니다. 
lat = df_31["위도"].mean()
long = df_31["경도"].mean()
lat, long

In [ ]:
folium.Map(
    location=(lat, long),
    control_scale=True,
)

In [ ]:
m = folium.Map([lat, long])

folium.Marker(
    [37.495593, 127.039032], 
    popup='<i>던킨도너츠</i>', 
    tooltip="던킨도너츠").add_to(m)

m

In [ ]:
df_31.index

In [ ]:
m = folium.Map([lat, long], zoom_start=12)

for i in df_31.index:
    
    sub_lat = df_31.loc[i, "위도"]
    sub_long = df_31.loc[i, "경도"]
    title = df_31.loc[i, "상호명"] + " - " + df_31.loc[i, "도로명주소"]
    
    icon_color = "blue"
    if df_31.loc[i, "브랜드명"] == "던킨도너츠":
        icon_color = "red"
    
    folium.Marker(
        [sub_lat, sub_long], 
        icon=folium.Icon(color=icon_color),
        popup=f'<i>{title}</i>', 
        tooltip=title).add_to(m)

m

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map([lat, long], zoom_start=12)
marker_cluster = MarkerCluster().add_to(m)

for i in df_31.index:
    
    sub_lat = df_31.loc[i, "위도"]
    sub_long = df_31.loc[i, "경도"]
    title = df_31.loc[i, "상호명"] + " - " + df_31.loc[i, "도로명주소"]
    
    icon_color = "blue"
    if df_31.loc[i, "브랜드명"] == "던킨도너츠":
        icon_color = "red"
    
    folium.Marker(
        [sub_lat, sub_long], 
        icon=folium.Icon(color=icon_color),
        popup=f'<i>{title}</i>', 
        tooltip=title).add_to(marker_cluster)

m.save('index.html')
m